# Example of a GFMAP full-extraction pipeline

Designing of GFMAP Job DataFrames and DataCube creators functions, as well as post job-actions.

Those dataframe should be containing all the necessary infromation to run a job and know where to save it.

### First step: splitting the job

Splitting the dataset of extraction in multiple job based on position is necessary to respect OpenEO limitations.

This script performs a split with the H3 hexagonal grid, yielding a list of sub-geodataframes.

A subtility here is that some polygons are not directly extracted (field with `extract=False`), but should be kept for post-job actions. This requirement is filled by removing sub-dataframes that do not contain any extractable polyons.

In [1]:
# Configuring the logging for the openeo_gfmap package
from openeo_gfmap.manager import _log
import logging

_log.setLevel(logging.DEBUG)

stream_handler = logging.StreamHandler()
_log.addHandler(stream_handler)

formatter = logging.Formatter('%(asctime)s|%(name)s|%(levelname)s:  %(message)s')
stream_handler.setFormatter(formatter)

# Exclude the other loggers from other libraries
class MyLoggerFilter(logging.Filter):
    def filter(self, record):
        return record.name == _log.name

stream_handler.addFilter(MyLoggerFilter())


In [2]:
from pathlib import Path
import geopandas as gpd
from openeo_gfmap.manager.job_splitters import split_job_hex

base_df_path = Path('/vitodata/worldcereal/tmp/kristof/GFMAP/2021_EUR_DEMO_POLY_110.gpkg')
base_df = gpd.read_file(base_df_path)
# Splits the job using GFMAP
split_jobs = split_job_hex(
    base_df, max_points=500, grid_resolution=4
)

print(f'{len(split_jobs)} jobs before filtering empty one (no extraction)')

# Remove the geometry where there are no points with the "extract" flag
split_jobs = [
    job for job in split_jobs if job.extract.any()
]
print(f'{len(split_jobs)} jobs after filtering empty one (no extraction)')


/data/users/Private/couchard/openeo-gfmap/src/openeo_gfmap/manager/job_splitters.py:53: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  polygons["h3index"] = polygons.geometry.centroid.apply(


137 jobs before filtering empty one (no extraction)
93 jobs after filtering empty one (no extraction)


### Second step: creating a dataframe for the GFMAP Job Manager

Implementing a function that yields a `pandas.DataFrame` where each row correponds to a job.

The dataframe should contain the informations required by the GFMAP Job Manager, as well as additional information used by the datacube creation function and the post-job action function.

The output dataframe should be savable as a .csv file.

Note: the full information of a sub-geodataframe of polygons can be saved into a row of a `pandas.DataFrame` by storing it in a row as string implementing the `geojson.FeatureCollection` interface. To convert the `geopandas.GeoDataFrame` into a stirng, simply use the `.to_json()` function.

In [3]:
from openeo_gfmap import Backend
from typing import List
import pandas as pd

def create_job_dataframe_s2(backend: Backend, split_jobs: List[gpd.GeoDataFrame]) -> pd.DataFrame:
    """Create a dataframe from the split jobs, containg all the necessary information to run the job."""
    columns = ['backend_name', 'out_prefix', 'out_extension', 'start_date', 'end_date', 'geometry']
    rows = []
    for job in split_jobs:
        # Compute the average in the valid date and make a buffer of 1.5 year around
        median_time = pd.to_datetime(job.valid_date).mean()
        start_date = median_time - pd.Timedelta(days=275)  # A bit more than 9 months
        end_date = median_time + pd.Timedelta(days=275)  # A bit more than 9 months
        
        rows.append(
            pd.Series(
                dict(zip(columns, [backend.value, 'S2', '.nc',  start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'), job.to_json()]))
            )
        )

    return pd.DataFrame(rows)

job_df = create_job_dataframe_s2(Backend.CDSE, split_jobs)

job_df

,backend_name,out_prefix,out_extension,start_date,end_date,geometry
0,cdse,S2,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
1,cdse,S2,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
2,cdse,S2,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
3,cdse,S2,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
4,cdse,S2,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
...,...,...,...,...,...,...
88,cdse,S2,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
89,cdse,S2,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
90,cdse,S2,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
91,cdse,S2,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."


In [4]:
# Run a subset of the jobs to test the manager, the selected jobs have a fair amount of geometries to extract
job_df = job_df.iloc[[0, 2, 3, -6]].reset_index(drop=True)
job_df

,backend_name,out_prefix,out_extension,start_date,end_date,geometry
0,cdse,S2,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
1,cdse,S2,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
2,cdse,S2,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
3,cdse,S2,.nc,2020-08-30,2022-03-03,"{""type"": ""FeatureCollection"", ""features"": [{""i..."


### Third step: implement the datacube creator function.

Implement a function to create, from the additional rows provided before, an `openeo.BatchJob` that will be used to run the job.

In this case we extract Sentinel-2 data, and we remove the polygons with `extract=False` (although we keep them in the row for the post-job action.)

In [11]:
import openeo

import pandas as pd
import geojson

from openeo_gfmap import TemporalContext, Backend, BackendContext, FetchType
from openeo_gfmap.fetching import build_sentinel2_l2a_extractor


def create_datacube_s2(row: pd.Series, connection: openeo.DataCube, provider=None, connection_provider=None) -> openeo.BatchJob:

    def buffer_geometry(geometry: geojson.FeatureCollection, buffer: float) -> str:
        gdf = gpd.GeoDataFrame.from_features(geometry).set_crs(epsg=4326)
        utm = gdf.estimate_utm_crs()
        gdf = gdf.to_crs(utm)
        gdf['geometry'] = gdf.centroid.buffer(distance=buffer, cap_style=3)

        gdf = gdf.to_crs(epsg=4326)
        return geojson.loads(gdf.to_json())


    def filter_extractonly_geometries(collection: geojson.FeatureCollection):
        # Filter out geometries that do not have the field extract=True
        features = [f for f in collection.features if f.properties.get('extract', False)]
        return geojson.FeatureCollection(features)

    start_date = row.start_date
    end_date = row.end_date
    temporal_context = TemporalContext(start_date, end_date)

    # Get the feature collection containing the geometry to the job
    geometry = geojson.loads(row.geometry)
    assert isinstance(geometry, geojson.FeatureCollection)

    # Filter the geometry to the rows with the extract only flag
    geometry = filter_extractonly_geometries(geometry)
    assert len(geometry.features) > 0, "No geometries with the extract flag found"

    # Performs a buffer of 64 px around the geometry
    geometry = buffer_geometry(geometry, 319)

    # Backend name and fetching type
    backend = Backend(row.backend_name)
    backend_context = BackendContext(backend)

    fetch_type = FetchType.POLYGON
    bands_to_download = ['S2-B01', 'S2-B02', 'S2-B03', 'S2-B04', 'S2-B05', 'S2-B06', 'S2-B07', 'S2-B08', 'S2-B8A', 'S2-B09', 'S2-B11', 'S2-B12', 'S2-SCL']

    # Create the job to extract S2
    extraction_parameters = {
        "target_resolution": 10
    }
    extractor = build_sentinel2_l2a_extractor(
        backend_context, bands=bands_to_download, fetch_type=fetch_type.POLYGON, **extraction_parameters 
    )

    cube = extractor.get_cube(connection, geometry, temporal_context)

    # Get the h3index to use in the tile
    h3index = geometry.features[0].properties['h3index']
    valid_date = geometry.features[0].properties['valid_date']

    # Increase the memory of the jobs
    job_options = {
        "executor-memory": "5G",
        "executor-memoryOverhead": "2G",
    }

    return cube.create_job(
        out_format="NetCDF",
        title=f"GFMAP_Extraction_S2_{h3index}_{valid_date}",
        sample_by_feature=True,
        job_options=job_options
    )


### Fourth step: create output paths

Implement a function that from a temporary path containing a job result, from the job dataframe row and the root folder will choose the output path where to save that job result.

In [6]:
%%time

# Load the S2 grid
s2_grid = gpd.read_file('./s2grid_bounds.geojson')

CPU times: user 4.62 s, sys: 40.8 ms, total: 4.67 s
Wall time: 4.67 s


In [7]:
from pathlib import Path
import xarray as xr
from pyproj import Transformer, CRS
from shapely.geometry import box, Point

def generate_output_path_s2(root_folder: Path, tmp_path: Path, geometry_index: int, row: pd.Series):
    features = geojson.loads(row.geometry)
    sample_id = features[geometry_index].properties['sample_id']
    ref_id = features[geometry_index].properties['ref_id']
    
    # Loads the array lazily in-memory
    try:
        inds = xr.open_dataset(tmp_path, chunks='auto')
        
        source_crs = CRS.from_wkt(inds.crs.attrs['crs_wkt'])
        dst_crs = CRS.from_epsg(4326)
        
        transformer = Transformer.from_crs(source_crs, dst_crs, always_xy=True)
        bounds = inds.x.min().item(), inds.y.min().item(), inds.x.max().item(), inds.y.max().item()
        bbox = box(*bounds)

        # Get the center of the box
        centroid = bbox.centroid
        lon, lat = transformer.transform(centroid.x, centroid.y)
        centroid_pt = Point(lon, lat)

        # Intersecting with the s2 grid
        intersecting = s2_grid.geometry.intersects(centroid_pt)

        # Select the intersecting cell that has a centroid the closest from the point
        intersecting_cells = s2_grid[intersecting]
        intersecting_cells['distance'] = intersecting_cells.distance(centroid_pt)
        intersecting_cells.sort_values('distance', inplace=True)
        s2_tile = intersecting_cells.iloc[0]

        s2_tile_id = s2_tile.tile

        subfolder = root_folder / ref_id / str(source_crs.to_epsg()) / s2_tile_id / sample_id
    except Exception:
        # TODO: _log.error('Could not find S2 tile for file, setting up a dummy path')
        subfolder = root_folder / 'unsortable'

    return subfolder / f'{row.out_prefix}_{sample_id}_{source_crs.to_epsg()}_{row.start_date}_{row.end_date}{row.out_extension}'
    

### Fifth step: Define the post-job action

The post-job action will be called once the job resut was downloaded and saved to a specific path.

A post-job action function must receive 3 parameters:
* `result_paths`: Paths to the downloaded job result files.
* `row`: The current job dataframe row.
* `parameters`: User-defined parameters set in the `GFMAPJobManager` constructor.

The post-job action must return a list of paths containing the results from that job. For example, if no file is created/deleted in the post-job action, then the user can simply return the list of paths it has received as input `result_paths`. If instead files are added or removed, then the user will need to modify this list accordingly before returning it.

In [8]:
from rasterio.features import rasterize
from rasterio.transform import from_bounds
import json

def post_job_action(result_paths: list, row: pd.Series, parameters: dict = {}) -> list:
    base_gpd = gpd.GeoDataFrame.from_features(json.loads(row.geometry)).set_crs(epsg=4326)
    assert len(base_gpd[base_gpd.extract == True]) == len(result_paths), "The number of result paths should be the same as the number of geometries"
    extracted_gpd = base_gpd[base_gpd.extract == True].reset_index(drop=True)
    # In this case we want to burn the metadata in a new file in the same folder as the S2 product
    for idx, result_path in enumerate(result_paths.copy()):
        sample_id = extracted_gpd.iloc[idx].sample_id
        ref_id = extracted_gpd.iloc[idx].ref_id
        lc_label = extracted_gpd.iloc[idx].landcover_label
        confidence = extracted_gpd.iloc[idx].confidence
        valid_date = extracted_gpd.iloc[idx].valid_date

        result_ds = xr.open_dataset(result_path, chunks='auto')

        target_crs = CRS.from_wkt(result_ds.crs.attrs['crs_wkt'])

        # Get the surrounding polygons around our extracted center geometry to rastetize them
        bounds = (result_ds.x.min().item(), result_ds.y.min().item(), result_ds.x.max().item(), result_ds.y.max().item())
        bbox = box(*bounds)
        surround_gpd = base_gpd.to_crs(target_crs).clip(bbox)

        # Burn the polygon croptypes
        transform = from_bounds(*bounds, result_ds.x.size, result_ds.y.size)
        croptype_shapes = list(zip(surround_gpd.geometry, surround_gpd.croptype_label))
        croptype = rasterize(croptype_shapes, out_shape=(result_ds.y.size, result_ds.x.size), transform=transform, all_touched=True, fill=0, default_value=65535, dtype='uint16')

        # Create the attributes to add to the metadata
        crs_layer = result_ds['crs']
        attributes = {
            'ref_id': ref_id,
            'sample_id': sample_id,
            'landcover_label': lc_label,
            'confidence': str(confidence),
            'valid_date': valid_date
        }
        attributes.update(result_ds.attrs)

        aux_dataset = xr.Dataset({'CROPTYPE': (('y', 'x'), croptype)}, coords={'y': result_ds.y, 'x': result_ds.x}, attrs=attributes)

        # Include the CRS layer from OpenEO
        aux_dataset['crs'] = crs_layer
        aux_dataset.attrs.update(result_ds.attrs)

        # Save the metadata in the same folder as the S2 product
        metadata_path = result_path.parent / f'AUX_{sample_id}_{target_crs.to_epsg()}_{valid_date}.nc'
        aux_dataset.to_netcdf(metadata_path, format='NETCDF4', engine='h5netcdf')
        result_paths.append(metadata_path)

    return result_paths


### Sixth and last step: Running the manager

Let's initialize and execute the Job Manager as defined the GFMAP, and then run it using the functions defined previously

In [9]:
from openeo_gfmap.manager.job_manager import GFMAPJobManager
from openeo_gfmap.backend import cdse_connection

base_output_dir = Path('/data/users/Public/couchard/world_cereal/extractions/')
tracking_job_csv = base_output_dir / 'job_tracker.csv'

manager = GFMAPJobManager(
    output_dir=base_output_dir,
    output_path_generator=generate_output_path_s2,
    post_job_action=post_job_action,
    poll_sleep=60,
    n_threads=2,
    post_job_params={}
)

manager.add_backend(
    Backend.CDSE.value, cdse_connection, parallel_jobs=2
)

In [10]:
manager.run_jobs(job_df, create_datacube_s2, tracking_job_csv)

2024-02-12 09:37:32,563|openeo_gfmap.manager|INFO:  Starting job manager using 2 worker threads.
2024-02-12 09:37:32,575|openeo_gfmap.manager|INFO:  Workers started, creating and running jobs.
2024-02-12 09:37:32,610|openeo_gfmap.manager|DEBUG:  Normalizing dataframe. Columns: Index(['backend_name', 'out_prefix', 'out_extension', 'start_date', 'end_date',
       'geometry', 'status', 'id', 'start_time', 'cpu', 'memory', 'duration',
       'description', 'costs'],
      dtype='object')
2024-02-12 09:37:32,612|openeo_gfmap.manager|DEBUG:  Updating status. 2 on 4 active jobs...


Authenticated using refresh token.


2024-02-12 09:37:33,882|openeo_gfmap.manager|DEBUG:  Status of job j-2402124f58244715803a335628079b9a is finished (on backend cdse).
2024-02-12 09:37:33,883|openeo_gfmap.manager|INFO:  Job j-2402124f58244715803a335628079b9a finished successfully, queueing on_job_done...
2024-02-12 09:37:33,888|openeo_gfmap.manager|DEBUG:  Worker thread Thread-4: polled finished job with status PostJobStatus.FINISHED.
2024-02-12 09:37:38,861|openeo_gfmap.manager|DEBUG:  Status of job j-24021279e49745e9a3806584fe0e8c9a is finished (on backend cdse).
2024-02-12 09:37:38,863|openeo_gfmap.manager|INFO:  Job j-24021279e49745e9a3806584fe0e8c9a finished successfully, queueing on_job_done...
2024-02-12 09:37:38,867|openeo_gfmap.manager|DEBUG:  Worker thread Thread-5: polled finished job with status PostJobStatus.FINISHED.


DataCube(<PGNode 'dimension_labels' at 0x7ffafb33a180>)


2024-02-12 09:37:43,664|openeo_gfmap.manager|DEBUG:  Downloading asset openEO_0.nc from job j-2402124f58244715803a335628079b9a -> /tmp/tmp91ksct_n
2024-02-12 09:37:45,148|openeo_gfmap.manager|DEBUG:  Downloading asset openEO_0.nc from job j-24021279e49745e9a3806584fe0e8c9a -> /tmp/tmp_4v2t_f0
2024-02-12 09:37:52,138|openeo_gfmap.manager|DEBUG:  Generating output path for asset openEO_0.nc from job j-2402124f58244715803a335628079b9a...
/tmp/ipykernel_23450/1596858655.py:32: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  intersecting_cells['distance'] = intersecting_cells.distance(centroid_pt)
/home/couchard/miniconda3/envs/gfmap/lib/python3.9/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

DataCube(<PGNode 'dimension_labels' at 0x7ffaf871cc70>)


2024-02-12 09:38:05,385|openeo_gfmap.manager|INFO:  Downloaded asset openEO_1.nc from job j-24021279e49745e9a3806584fe0e8c9a -> /data/users/Public/couchard/world_cereal/extractions/2021_EUR_DEMO_POLY_110/32633/33TXN/at2021lpis1313057/S2_at2021lpis1313057_32633_2020-08-30_2022-03-03.nc
2024-02-12 09:38:05,390|openeo_gfmap.manager|DEBUG:  Downloading asset openEO_2.nc from job j-24021279e49745e9a3806584fe0e8c9a -> /tmp/tmpkew_9hyq
2024-02-12 09:38:08,255|openeo_gfmap.manager|DEBUG:  Generating output path for asset openEO_2.nc from job j-2402124f58244715803a335628079b9a...
/tmp/ipykernel_23450/1596858655.py:32: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  intersecting_cells['distance'] = intersecting_cells.distance(centroid_pt)
/home/couchard/miniconda3/envs/gfmap/lib/python3.9/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
